<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/35_geemap_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

#preliminary set

In [ ]:
import pandas as pd
import gzip
import urllib
from urllib.request import urlopen
import requests
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = 'drive/My Drive/Colab Notebooks/4月/'

#download data

In [ ]:
def load_data(month,start_day,end_day, outpath):
  if month <10:
    mon = str(0) + str(month)
  else:
    mon = str(month)
  #path = str(storage_path)

  for k in range(start_day,end_day+1):
    if k <=9:
        day= str(0) + str(k)
    else:
        day= str(k)
    for i in range(0,24):
        if i <=9:
            hours = str(0)+str(i)
        else:
            hours = str(i)
        for j in range(0,60):
            if j <=9:
                min = str(0) + str(j)
            else:
                min = str(j)
            u = 'https://tisvcloud.freeway.gov.tw/history/motc20/VD/2021'+ mon + day + '/VDLive_' + hours + min + '.xml.gz'
            
            try:
                urlopen(u)
            except:
                print(mon,day,hours,min,'is not found')
                print('-------------------------------')
                pass
            else:
                file_name = u.split("/")[-1]
                urllib.request.urlretrieve(u, file_name)
  


                f=gzip.open(file_name,'r')
                xmldata = f.read()
                f.close()
                print (mon,day,hours,min,'is downloaded')
                with open('temp' +'.xml', 'wb') as file:
                    file.write(xmldata)


                out = path + mon + day + hours + min + '.csv'
                xml_to_csv('temp.xml',out)
                print('-------------------------------')

#convert xml to csv

In [ ]:
import pandas as pd 
from xml.dom import minidom
import numpy as np

In [ ]:
def xml_to_csv(filename,output_path):
  xmldoc = minidom.parse(filename)
  VDID = xmldoc.getElementsByTagName('VDID')
  LinkID = xmldoc.getElementsByTagName('LinkID')
  LaneID = xmldoc.getElementsByTagName('LaneID')
  Speed = xmldoc.getElementsByTagName('Speed')
  Occupancy = xmldoc.getElementsByTagName('Occupancy')
  Volume = xmldoc.getElementsByTagName('Volume')

  #VDID+linkID
  cols1 = ["VDID", "LinkID"] 
  rows1 = []

  for i in range(len(VDID)):
    rows1.append({'VDID':VDID[i].firstChild.data,
           'LinkID':LinkID[i].firstChild.data})
  df1 = pd.DataFrame(rows1, columns=cols1)

  #Speed, occupancy, volume
  cols2 = ["Speed", "Occupancy" ,'Volume'] 
  rows2 = []

  temp_occupancy = []
  temp_speed = []
  temp_pcu = []
  count= 0

  for j in range(len(LaneID)-1):
    #計算車當量
    a,b,c = 3*j,3*j+1,3*j+2
    pcu = int(Volume[a].firstChild.data)+(int(Volume[b].firstChild.data))*1.5+(int(Volume[c].firstChild.data))*2
    #如果現在跟下一組都0直接輸出
    if int(LaneID[j].firstChild.data)==0 and int(LaneID[j+1].firstChild.data) == 0 :
      temp_occupancy.append(int(Occupancy[j].firstChild.data))
      temp_speed.append(int(Speed[j*4].firstChild.data))
      temp_pcu.append(pcu)

      occ , spd , vol = np.mean(temp_occupancy) , np.mean(temp_speed) , np.mean(temp_pcu)
      temp_occupancy, temp_speed, temp_pcu = [],[],[]

      #print('佔有率:' , occ , '速度:' , spd , '當量:' , vol)
      rows2.append({'Speed':spd, 'Occupancy':occ,'Volume':vol})
      count +=1

    #如果下一組比較大就加
    elif int(LaneID[j+1].firstChild.data) > int(LaneID[j].firstChild.data):
      temp_occupancy.append(int(Occupancy[j].firstChild.data))
      temp_speed.append(int(Speed[j*4].firstChild.data))
      temp_pcu.append(pcu)

    #如果下一組比較小就結束 取平均 歸零
    elif int(LaneID[j+1].firstChild.data) < int(LaneID[j].firstChild.data):
      temp_occupancy.append(int(Occupancy[j].firstChild.data))
      temp_speed.append(int(Speed[j*4].firstChild.data))
      temp_pcu.append(pcu)

      occ , spd , vol = np.mean(temp_occupancy) , np.mean(temp_speed) , np.mean(temp_pcu)
      temp_occupancy, temp_speed, temp_pcu = [],[],[]

      #print('佔有率:' , occ , '速度:' , spd , '當量:' , vol)
      rows2.append({'Speed':spd, 'Occupancy':occ,'Volume':vol})
      count +=1

  #最後一組的輸出
  j=len(LaneID)-1
  a,b,c = 3*j,3*j+1,3*j+2
  pcu = int(Volume[a].firstChild.data)+(int(Volume[b].firstChild.data))*1.5+(int(Volume[c].firstChild.data))*2
  temp_occupancy.append(int(Occupancy[j-1].firstChild.data))
  temp_speed.append(int(Speed[(j-1)*4].firstChild.data))
  temp_pcu.append(pcu)

  occ , spd , vol = np.mean(temp_occupancy) , np.mean(temp_speed) , np.mean(temp_pcu)

  temp_occupancy, temp_speed, temp_pcu = [],[],[]
  count +=1
  rows2.append({'Speed':spd, 'Occupancy':occ,'Volume':vol})
  #print('佔有率:' , occ , '速度:' , spd , '當量:' , vol)
  #print('總數:',count)

  df2 = pd.DataFrame(rows2, columns=cols2) 
  

  #concat and save csv file
  final = pd.concat([df1,df2],axis=1)
  final.to_csv(output_path,encoding="utf_8_sig")
  print('Convert is done')     

#run

In [ ]:
load_data(4,1,30,path)

串流輸出內容已截斷至最後 5000 行。
04 03 15 10 is downloaded
Convert is done
-------------------------------
04 03 15 11 is downloaded
Convert is done
-------------------------------
04 03 15 12 is downloaded
Convert is done
-------------------------------
04 03 15 13 is downloaded
Convert is done
-------------------------------
04 03 15 14 is downloaded
Convert is done
-------------------------------
04 03 15 15 is downloaded
Convert is done
-------------------------------
04 03 15 16 is downloaded
Convert is done
-------------------------------
04 03 15 17 is downloaded
Convert is done
-------------------------------
04 03 15 18 is downloaded
Convert is done
-------------------------------
04 03 15 19 is downloaded
Convert is done
-------------------------------
04 03 15 20 is downloaded
Convert is done
-------------------------------
04 03 15 21 is downloaded
Convert is done
-------------------------------
04 03 15 22 is downloaded
Convert is done
-------------------------------
04 03 15 23 is do

KeyboardInterrupt: ignored